# Elastic Stackの構築：概要
----
Elastic Stackの概念やNodeの種類、 サイジングの観点について説明します。

第1節はアーキテクチャーの説明でドキュメントのみで実行すべき手順は含まれていません。第2節は、この後の構築作業で利用する設定ファイルを生成します。

**<font color="red">デモ環境では収容を変更することはできないので1節を読み飛ばし、2節から実行しても問題ありません。</font>**

## 基本概念
実際の構築の前に、クラスタやそれを構成するNodeに関する基本概念を説明します。

### Elasticsearchクラスタ

Elasticsearchは複数台のサーバで起動しているElasticsearch間で連携して、分散処理を行うことができます。  
以下のような要素によって構成されます。

![images/01_basic_concept.png](images/01_basic_concept.png)

**※Node数が3、Shard数が3、Replica数が1の場合の例**

|名称|意味|
|----|----|
|ノード<br>Node|動作しているElasticsearch Serverのことです。|
|クラスタ<br>Cluster|Nodeの集合です。|
|インデックス<br>Index|リレーショナルデータベースでのDatabaseに相当する概念です。<br>Indexは複数のType(テーブルに相当)を持ち、<br>Typeは複数のDocument(レコードに相当)を持ちます。<br>1つのClusterに複数のIndexを作成できます。|
|シャード<br>Shard|Indexを分割した断片です。Shardの単位でNodeに配置されます。|
|プライマリシャード<br>Primary Shard|読み書きの対象となるShardです。|
|レプリカシャード<br>Replica Shard|読み取り専用の、Primary Shardの複製です。|


複数のNodeを生成してそれぞれを物理マシン等に分散配置することで、蓄積するデータ量や検索の速度を向上できますが、
用意した各物理マシンのスペックを十分に発揮させてElasticsearchのクラウドを稼働させるには、蓄積しているデータも分割して分散配置する必要があります。  
Shardはそのための分割の単位です。分割するShard数を設定することで、自動的にCluster内のNodeにPrimary Shardとして配置されます。  
さらに設定されたReplica数だけ、各Primary Shardが複製され、可用性を確保します。

### Nodeの種類と使い分け

Nodeの役割はデータの蓄積だけではありません。Clusterの管理やデータの加工などの役割を持った、複数のNodeの種類が存在します。  
種類ごとの役割は次の通りです。

![images/01_node_type.png](images/01_node_type.png)

| 種別              | 役割                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | elasticsearch.ymlの設定内容 |
|-------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------| |
| Coordinating Node | 検索リクエストやインデクシングリクエストなどを受けつけるノードです。<br>このノードは、リクエストを受けた際に必要なデータを持っているノードに対してリクエストを転送し、返ってきた結果をマージしてからレスポンスとして返す役割を持ちます。|**※特になし**<br><br>全てのノードは、設定によらずCoordinating Nodeとして動作します。実際のどのノードをCoordinating Nodeとするかは利用方法次第となります。 |
| Master Node       | クラスタの管理を行うノードです。このノードはクラスタに属するノードを把握したり、データを保持するシャードを決定するなど、クラスタ全体に関わる役割を持ちます。<br>右の設定をしたノードはMaster Eligible Nodeと呼ばれるMasterの候補となります。<br>それらの候補が連携し、うち1台がMaster Nodeとして選出されるよう動きます。 |**node.master=true**<br>(デフォルトはtrue)<br><br>つまりデフォルトでMaster Eligible Nodeとなります。<br>なお、特にData nodeの役割を持たずにMaster Eligible Nodeの役割のみを持つノードをDedicated Master Nodeと呼びます。 |
| Data Node         | Elasticsearchのデータを保持するノードです。このノードはデータを保持し、クエリに対応した結果を返す役割を持ちます。<br><br>Data NodeはMaster Nodeとは兼任することも、別にすることもできます。<br>特にノード数の少ないクラスタ構成（～5台程度）では、すべてのノードをMaster NodeかつData Nodeとし、ノード数が増えた場合には役割を分けると良いでしょう。 |**node.data=true**<br>(デフォルトはtrue)<br><br>つまりデフォルトでData Nodeとなります。 |
| Client Node       | Master Node、Data Nodeのいずれの役割も担わず、Coordinating Nodeの役割に特化したノードの便宜的な呼び方です。<br><br>Kibanaを配置したサーバにElasticsearchのClient nodeを配置することで、他のElasticsearchのクラスタに負荷を掛けないようにしたり、Elasticsearchのロードバランサのように振る舞わせるようにしたりなどの用途で用います。|**node.master=false**<br>**node.data=false** |
| Ingest Node       | 実際のインデクシング処理を行う前に、ドキュメントを加工することができるノードです。<br>いくつかの加工処理を記述した「pipeline」を登録しておけば、その内容に従った処理をすることが可能です。<br>また、pipelineの振る舞いを検査するためのSimulate Pipeline APIも提供されています。 | **node.ingest=true**<br>(デフォルトはtrue)<br><br>つまりデフォルトでIngest Nodeとなります。 |

### クラスタ構成の例

クラスタをサーバに配置する例を示します。
Data Node3台、Master Node1台、Client Node1台です。  

それと別にLogstash用のサーバを1台設けています。  
Logstashとは、Elastic Stackをを構成するプロダクトの１つです。  
様々な形式のデータの読み込み、加工、格納を処理するパイプラインを簡単に構築できます。  

![images/01_5server.png](images/01_5server.png)

この図の例では、Master Node、Client Node、Logstashは、Data Node群とは別のサーバを確保していますが、実際はサーバ内に同居させることも可能です。  
どのような構成とするかはシステムの要件に応じて調整すべきポイントとなります。

次の章ではサーバの構成やサーバのスペック等、サイジングの観点について説明します。



## サイジングの考え方
構築にあたり、1サーバのスペック、設定するshardの数、各役割のノード数を決める必要があります。  
その際の指針となる考えを示します。  


具体的な数値に関しては用途に応じて決定してください。

### 1サーバ当たりのスペック

Elasticsearchに何をさせるかによってボトルネックになる箇所が異なるため、  
ハードウェアのスペックを決定する際に、ある程度「どういう処理をするのか」を想定しておく必要があります。  

Elasticsearchを利用する際に特に検討が必要なスペックは以下の通りです。  

- CPU: 全体の処理速度とコア数はクエリの並列処理  
- ディスク: 検索とindexの頻度(データ登録の頻度)  
- メモリ: キャッシュとして利用されるため、ディスクと同じく検索のレスポンスに影響します。  

#### CPUのコア数

CPUのコア数は検索のスループットに影響します。  
コア数に応じて複数の検索リクエストを並行で処理できる機会が増えるためです。

#### ディスクサイズ

蓄積できるデータ量に影響します。  
またReplica Shardの仕組みを使って他のノードのデータのバックアップを持ち合う場合は、より大きなディスクサイズを必要とします。

最低限、必要なディスクサイズは以下の計算式で求められます。

**データサイズ ÷ Data Node数 × (１＋Replica数）**

これに安全率や、同サーバ内の他のプロセスが利用するサイズなどを勘案してディスクサイズを決定してください。

#### メモリサイズ

検索のレイテンシに影響します。検索データのキャッシュとしてメモリが利用されるためです。

基本的にメモリが多ければ多いほど、キャッシュヒット率は上がり、応答が速くなる可能性は高まりますが、  
メモリ量によって検索が速くなるかについては、発行されるクエリの種類にどれほどのバラつきがあるかによる（バラつきが大きいほどキャッシュヒットする確率は低くなる）ため、  
同じ応答速度を保証したい場合でもメモリサイズが異なる場合があります。


単純な検索クエリについてElastic社が性能検証を行った結果、もっともパフォーマンスが出やすいのが「メモリ:ディスク=1:16」と実測されました。  
例えば、メモリが64GBの場合には1TBのデータ保持が適切です。  

※メモリとディスク容量の比率はElastic社の資料[Sizing Scenario](https://speakerdeck.com/elastic/quantitative-cluster-sizing?slide=16)を参考にしています。

|メモリ(GB)|ディスク(GB)|
|---------|-----------|
| 32      | 512       |
| 64      | 1024      |
<center>メモリとディスクの割合を1:16で計算した場合</center>

ただし、、そもそも検索速度をそれほど必要とせず、蓄積したいデータ量が多い場合は、必要なメモリの比率はさらに下がります。

性能要件と費用対効果を勘案して、実測によりメモリサイズを調整してください。  
ノード1台あたりのメモリサイズの詳細はElasticsearch Referenceの[Memory](https://www.elastic.co/guide/en/elasticsearch/guide/current/hardware.html#_memory)を参照してください。

### Shard数

検索のレイテンシに影響します。  
分割されたShardがData Nodeに分散配置されていれば、Data Nodeの数に応じた並列処理が期待できます。必要な性能要件と実測に応じて、Data Nodeの数に合わせてShard数を設定してください。

設定方法としてshard数を縮小するためのAPIが用意されています。  
詳細はElasticsearch Referenceの[Shrink Index](https://www.elastic.co/guide/en/elasticsearch/reference/current/indices-shrink-index.html)を参照してください。

また、shard数はクエリ解析時にも影響があります。  
詳細は[01_05_Diagnostics](01_05_Diagnostics.ipynb#Slowログが無いか？-127)を参照してください。

### Replica数

Data Nodeが故障した場合の耐障害性に影響します。   
Replicaが残っていれば、Data Nodeがクラッシュした際に動作を継続可能です。  
Replica数がある分だけ動作継続の可能性は高まりますが、Data Nodeと同数以上の数を設定しても意味はありません。それ未満の数を設定してください。

また[ディスクサイズ](#ディスクサイズ)で説明した通り、Replica数が大きくなるほど必要となるディスクサイズは大きくなります。  
耐障害性と必要サイズのトレードオフを勘案してReplica数を設定してください。

### サーバ台数

求められる検索速度、扱うデータ量、用意できるサーバスペックによってクラスタを構成する台数は大きく変化します。  
そのため、正確な見積もりは難しいため、ベンチマークを実施することを推奨します。  
ここでは、データ、マスターノード数を決定する上での考えを示します。

#### Data Node数

Data Node数は、蓄積できるデータサイズ、耐障害性、検索のレイテンシに影響します。  
それら3つの観点を元に、要件に応じて調整してください。

__(1) 蓄積できるデータサイズ__

前述の[ディスクサイズ](#ディスクサイズ)にもありますが、1台あたりのディスクサイズとData Node数の掛け算で蓄積できるデータサイズが決まります。  

**(2) 耐障害性**

[Shard数](#Shard数)で設定したReplica数の設定によって、Data Nodeに障害が発生した場合に残ったサーバで動作を継続できます。  
そのためにはReplcaを設定した数、別のData Nodeに配置する必要がありますので、  
**1＋Replica数**  
分のData Nodeが必要になります。

**(3)検索のレイテンシ**

Data Nodeの数に応じて検索クエリを分散処理できます。  
実際は、分割された各々のShardの分だけクエリを分散することになるので、  
(Primary)Shard数と同数以上のData Nodeを用意することで、分散処理が最大化されます。


#### Master Node数

Master Node数は耐障害性に影響します。正確には、Master Eligible Node(Master Nodeの候補)の数が影響します。

Split brain問題を避けるために3台以上、かつ奇数台とすることが望ましいです。  
Split brainとは、クラスタ内でMaster nodeが複数存在してしまいデータが失われる危険性がある問題です。  
詳細はElasticsearch Referenceの[Master Eligible Node](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-node.html#split-brain)を参照してください。

また、Master NodeをData Nodeに兼任させるか検討してください。  
兼任の場合、Master Nodeとして動作する分の負荷がData Nodeの処理を阻害しますが、  
実測上で問題が無ければ、サーバ台数を節約する観点でそのままにすることは有効です。

#### Client Node数

Client Node（Master Node、Data Nodeのいずれの役割も担わず、Coordinating Nodeの役割に特化したノード）を設けるかについて検討します。  

Client Nodeが無い場合は、Data Nodeがリクエストを処理しますが、その処理の重さで検索に悪影響が出ている場合、Client Nodeを立てることで負荷を軽減することができる可能性があります。  
実測により判断してください。

#### Logstashサーバ数

他のElasticsearchのNodeと同居させるかどうかが、データ書き込みのスループットに影響します。  
独立したサーバにLogstashを切り出すことで、スループットが向上します。

なお、冗長化の要件で複数のLogstashサーバを立てる場合、  
Logstash自身には他のインスタンスと協調してデータを分配する機能は無いため、  
前段で分散メッセージングシステムである[Apache Kafka](http://kafka.apache.org/)などの、  
データをロードバランシングするシステムが必要になります。


### 本体のパラメータ

#### ヒープサイズ

ヒープサイズについて、Elasticsearch: The Definitive Guideの[Heap: Sizing and Swapping](https://www.elastic.co/guide/en/elasticsearch/guide/current/heap-sizing.html)では、次の2つの条件を両方とも満たすことが推奨されています。

1. マシンのメモリの半分とすること  
  - 残りの半分をApache Lucene（Elasticsearchが内部で利用している全文検索エンジン）が利用するためです。   
  Luceneは検索対象となるデータのファイルをOSによってメモリにキャッシュさせることで高速な検索を実現しています。  
  このキャッシュ用に、ヒープがマシンのメモリを占有しきらないようにする必要があります。
2. 32GB未満とすること
  - Java VMがヒープ上のオブジェクトを高速に扱うための仕組みが、32GBを超えると働かなくなるためです。

詳細は[Heap: Sizing and Swapping](https://www.elastic.co/guide/en/elasticsearch/guide/current/heap-sizing.html)を参照してください。


#### 最小Master Node数

最小Master Node数とは、クラスタに参加しているMaster Eligible Nodeが障害等で減った際に、最小で何台までであればクラスタとして動作を継続させるかを示す数です。  
数としては、  

**（Master Eligible Node数 / 2）＋ 1**  

を設定します。  
つまり**Master Eligible Node数の過半数**を設定します。  
例えばMaster Eligible Nodeが3台であれば2、5台であれば3を設定します。

理由については、[Master Node数](#Master-Node数)でもリンクした、Elasticsearch Referenceの[Avoiding split brain with minimum_master_node](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-node.html#split-brain)に説明があります。  

簡単に述べると、Split brain問題とは、Master Eligible Node群がネットワーク障害等で2つに分断された際に、それぞれがクラスタとしてデータを扱ってしまうと、障害が解決して再統合された際にデータが失われてしまう問題です。  
最小Master Node数はこれを解決するために設定します。つまり過半数（＝最小Master Node数）のMaster Eligible Nodeに満たない側の分断クラスタはMasterになれないようにすることで、分断があっても高々1つの分断クラスタ（内のMaster）しか動作できないようにします。  

例えば5台のMasterが3台と2台に分断された場合は、最小マスタノードが3なので、3台の側のクラスタでしかMasterが選出されません。  
よってそちらだけが動き続け、5台に再統合されたときもデータの喪失が防げます。

最小マスタノード数はelasticsearch.ymlの**discovery.zen.minimum_master_nodes**に設定します。

#### その他のパフォーマンス改善のためのパラメータ

パフォーマンスを改善するための主なパラメータは次のようなものになります。  
設定にはindexごとの設定や、ノードごとの設定、クラスタ全体の設定があり、設定方法が異なります。  
設定に関しての詳細はElasticsearch Referenceの[Configuring Elasticsearch](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/settings.html)を参照してください。

| チューニング項目         | 設定項目名            | チューニング概要       |詳細の参照先       |設定方法                                                                                                                                                                                                      |
|--------------------------|-----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------|
| [indexの更新間隔](#1サーバ当たりのスペック)          | refresh_interval      | indexの更新間隔を広げることで、インデックス処理は遅延しますが、Elasticsearchの負荷を下げることができます。|[Index Modules](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/index-modules.html#dynamic-index-settings)         |indexごとに設定することができます。REST APIから設定します。                                                                                                                                            |
| [リクエストキューのサイズ](#サーバ台数の決め方) | thread_pool.queue.size | リクエストキューのサイズを増やすことで、高負荷時のエラー発生を防ぐことができます。<br>キューが溢れた場合、Elasticsearchはドキュメントを破棄し、429エラーを返します。<br>Logstashを用いてドキュメント登録を行っている場合、下記のようなエラーが出力されます。<br>その場合はキューのサイズを増やすか、Elasticsearchの処理速度を向上させる必要があります。<br><br>17:42:11.840 [[main]>worker6] INFO  logstash.outputs.elasticsearch - retrying failed action with response code: 429 ({"type"=>"es_rejected_execution_exception", "reason"=>"rejected execution of org.elasticsearch.transport.TransportService$6@1adec92 on EsThreadPoolExecutor[bulk, queue capacity = 0, org.elasticsearch.common.util.concurrent.EsThreadPoolExecutor@75905a2[Running, pool size = 1, active threads = 0, queued tasks = 0, completed tasks = 1]]"})|[Thread Pool](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-threadpool.html)             |elasticsearch.ymlで設定することができます。                                                                                                                                                        |
| [バルク処理のスレッド数](#1サーバ当たりのスペック)   | thread_pool.bulk.size  | バルクインポート処理（複数のドキュメントをまとめて追加する処理）を受け付けるスレッド数を増減させます。<br>スレッド数を増加させることによって処理が並列化され、パフォーマンスの向上が期待されます。<br>ただし、稼働マシンのコア数を超える値を設定できないように制限されています。|[Thread Pool](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-threadpool.html)   |elasticsearch.ymlで設定することができます。                                                                                              

# 収容を決め設定ファイルを出力する

Elasticsearchを構築するにあたり、どのようなクラスタ構成にするかを決定する必要があります。

## 典型的な構成例

お試し用構成は、1台のサーバに全ての機能を投入した構成です。冗長性もパフォーマンスも考慮していないため運用には適しませんが、Elasticsearch自体を試したい場合に利用可能です。  
運用可能最小構成と運用可能高性能構成は、扱えるデータサイズや前の章のサイジング観点に応じて選んでください。

**<font color="red">本デモ環境では、お試し用構成(データ1TB)のみ設定可能です。</font>**

|設定項目|お試し用構成<br>(データ1TB)|運用可能最小構成<br>(データ1.5TB)|運用可能高性能構成<br>(データ2.5TB)|
|---|--:|--:|--:|
|**1サーバ当たりのスペック**||||
|　　CPUのコア数|8|8|8|
|　　ディスクサイズ(GB)|1024|3072|5120|
|　　メモリサイズ(GB)|64|64|64|
|**Shard数**|1|3|5|
|**Replica数**|0|1|1|
|**サーバ台数**||||
|　　Data Node数|1|3|5|
|　　Master Node数(※1)|1|3|3|
|　　Client Node数|0|0|0|
|　　Logstashサーバ数(※1)|1|1|1|
|**本体のパラメータ**||||
|　　ヒープサイズ(GB)|31|31|31|
|　　最小マスタノード数|1|2|2|


※1）この次の章で上の構成を出力させますが、Master NodeやLogstashは全てData Nodeと同居させる構成としています。必要に応じて出力結果を修正してください。

## 設定ファイルの出力

この後の「収容」「インストール」のNotebookで用いるパラメータを生成します。

Ansibleに環境構築をおこなわせるため、以下の2つのファイルを更新します。
- hosts ... インベントリ　グループとホストの対応関係
- group_vars ... Ansibleが各サーバに渡すパラメータ

ただしhostsは、このNotebookではひな型のみを生成します。  
これは01_02_Accommodation_XXX.ipynbでサーバインスタンスを生成しないとIPアドレスが定まらないためです。  
実際のIPアドレスを当てはめたhostsファイルの生成は01_03_Install.ipynbにて実施します。

まず出力に含めるクラスタ名を指定します。  
**同じネットワーク内で複数のクラスタを作成する場合、クラスタ名が重複しないように注意してください。**  
同一クラスタだとみなされると、予期しないサーバがクラスタに併合されてしまいます。

In [1]:
#クラスタ名
CLUSTER_NAME = 'es-cluster'

**前の章の[典型的な構成](#典型的な構成)に示した構成のうちどれを利用するかを、下のセルを実行して現れるプルダウンから選択し、  
「設定ファイル出力」ボタンを押してください。  **


選択した内容でAnsible用のhostsのテンプレート、およびgroup_varsが出力されます。

In [2]:
import os
if not os.path.isdir('group_vars'):
    os.mkdir('group_vars')

%run scripts/get-struct-selector.py
show_selector(CLUSTER_NAME)

**<font color="red">本デモ環境では、お試し用構成(データ1TB)のみ設定可能です。</font>**



### 内容の確認・修正
出力したファイルの内容を確認、修正したり、前回のバージョンと比較したい場合は以下から実施できます。

出力されたhosts_template、group_vars/all のファイルを修正したい場合は、次のリンクから編集してください。

- **[hosts_template](../edit/hosts_template)**
- **[group_vars/all](../edit/group_vars/all)**

出力結果を確認するには、次のセルを実行してください。

In [3]:
!cat hosts_template

In [4]:
!cat group_vars/all